In [64]:
import rioxarray as rio
from rioxarray.merge import merge_arrays
import numpy as np

In [2]:
xds_0 = rio.open_rasterio("./dems/larimerAreaDEM.tif")
xds_1 = rio.open_rasterio('./dems/rightOfLarimer.tif')
xds_2 = rio.open_rasterio('./dems/northEastoFLarimer.tif')
xds_3 = rio.open_rasterio('./dems/above_larimer.tif')

In [3]:
geohashLength = 4

In [4]:
import geohash

In [5]:
geohash.bbox('9xj6')

{'s': 39.7265625, 'w': -105.1171875, 'n': 39.90234375, 'e': -104.765625}

In [6]:
def get_geohash_adjacent(gh, direction='e'):
    if not direction in 'nsew':
        raise Exception(f"{e} not in 'nsew'. This is not a valid direction")
    neighbors = geohash.neighbors(gh)
    m = {
        'w': 0,
        'e': 1,
        's': 2,
        'n': 5
    }
    return neighbors[m[direction]]

In [7]:
get_geohash_adjacent('9xh', 'e')

'9xj'

In [51]:
def tld_bounds_and_geohash_bounds_intersect(tld_bounds, gh_bounds):
    #intersect case
    if tld_bounds[0] < gh_bounds['w'] < tld_bounds[2] or tld_bounds[0] < gh_bounds['e'] < tld_bounds[2]:
        if tld_bounds[1] < gh_bounds['s'] < tld_bounds[3] or tld_bounds[1] < gh_bounds['n'] < tld_bounds[3]:
            return True
    return False

In [52]:
def convert_tld_to_int16_wgs84_simple(tld):
    tld_wgs = tld.rio.reproject("wgs84")
    return tld_wgs[0].astype('int16')

In [10]:
tlds = list(map(convert_tld_to_int16_wgs84_simple, [xds_0,xds_1,xds_2,xds_3]))

In [58]:
def slice(t, x_0=float('-inf') , x_1=float('inf'), y_0=float('inf'), y_1=float('-inf')):
    return t.rio.slice_xy(x_0,y_1,x_1,y_0)

In [59]:
def CASE_1(t, gh_bounds):
    return slice(t, gh_bounds['w'], gh_bounds['e'], gh_bounds['n'], gh_bounds['s'])

In [91]:
def CASE_GT1(tlds, gh_bounds):
    slices = list(map(lambda tld: slice(tld, gh_bounds['w'], gh_bounds['e'], gh_bounds['n'], gh_bounds['s']), tlds))
    print(f"MERGING {len(slices)} SLICES")
    return merge_arrays(slices, nodata=-9999)

In [92]:
def build_dem_within_geohash(gh, tlds):
    gh_bounds = geohash.bbox(gh)
    intersecting_tlds = list(filter(lambda tld: tld_bounds_and_geohash_bounds_intersect(tld.rio.bounds(), gh_bounds), tlds))
    if len(intersecting_tlds) == 1: #CASE 1
        print("CASE 1")
        return CASE_1(intersecting_tlds[0], gh_bounds)
    elif len(intersecting_tlds) > 1:
        print("CASE GT1")
        return CASE_GT1(intersecting_tlds, gh_bounds)
     

In [93]:
build_dem_within_geohash('9xm3', tlds)

CASE GT1
MERGING 4 SLICES


/home/dr101/.local/lib/python3.8/site-packages/rioxarray/raster_writer.py:108: UserWarning: The nodata value (-999999.0) has been automatically changed to (-16959) to match the dtype of the data.
  warnings.warn(


<xarray.DataArray (y: 1898, x: 3798)>
array([[ 2042,  2042,  2042, ...,  1831,  1831, -9999],
       [ 2042,  2042,  2042, ...,  1832,  1831, -9999],
       [ 2042,  2042,  2042, ...,  1832,  1831, -9999],
       ...,
       [ 1953,  1952,  1951, ...,  1838,  1837, -9999],
       [ 1955,  1954,  1952, ...,  1840,  1840, -9999],
       [ 1957,  1955,  1954, ...,  1841,  1841, -9999]], dtype=int16)
Coordinates:
  * x            (x) float64 -105.1 -105.1 -105.1 ... -104.8 -104.8 -104.8
  * y            (y) float64 41.13 41.13 41.13 41.13 ... 40.96 40.96 40.96 40.96
    band         int64 1
    spatial_ref  int64 0
Attributes:
    LAYER_TYPE:                athematic
    RepresentationType:        ATHEMATIC
    STATISTICS_MAXIMUM:        4347.1943359375
    STATISTICS_MEAN:           2442.9052512532
    STATISTICS_MINIMUM:        1473.9787597656
    STATISTICS_STDDEV:         614.85092582736
    STATISTICS_VALID_PERCENT:  100
    scale_factor:              1.0
    add_offset:                0.0
    long_name:                 Layer_1
    _FillValue:                -9999